In [27]:
from pathlib import Path
import datetime
from zoneinfo import ZoneInfo
from datetime import datetime
import os
import re
import scipy.io as sio
import numpy as np
import pandas as pd

import pynwb
from pynwb import NWBHDF5IO, NWBFile

import manimoh_utils as mu
import manimoh_nwb_converters as mnc

In [ ]:
# Import your custom extension
from ndx_mvdmlab_metadata import (
    LabMetaDataExtension, 
    ProbeExtension, 
    OdorantInfoExtension, 
    ExperimentalBlockExtension, 
    PreprocessedAnnotationExtension
)

In [ ]:
def add_lab_metadata_to_nwb(nwbfile, session_metadata):
    '''
    Function to add lab-specific metadata to the NWB file.
    '''
    # Create probe extension for probe 1
    probe1_extension = None
    if 'probe1_ID' in session_metadata:
        probe1_metadata = {}
        for key in session_metadata:
            if 'probe1_' in key and 'ID' not in key and 'location' not in key:
                if 'Depth' not in key:
                    probe1_metadata[key.split('_')[-1]] = session_metadata[key]
                else:
                    probe1_metadata['depth'] = session_metadata[key]

        probe1_extension = ProbeExtension(
            name='probe1',
            ID=session_metadata.get('probe1_ID'),
            **probe1_metadata
        )

    # Create probe extension for probe 2 (if exists)
    probe2_extension = None
    if 'probe2_ID' in session_metadata:
        probe2_metadata = {}
        for key in session_metadata:
            if 'probe2_' in key and 'ID' not in key and 'location' not in key:
                if 'Depth' not in key:
                    probe2_metadata[key.split('_')[-1]] = session_metadata[key]
                else:
                    probe2_metadata['depth'] = session_metadata[key]
        
        probe2_extension = ProbeExtension(
            name='probe2',
            ID=session_metadata.get('probe2_ID'),
            **probe2_metadata
        )

    # Create odorant info extension
    odorant_info = {}
    for odor in ['A', 'B', 'C', 'D', 'E', 'F', 'G']:
        odor_field = f'odor{odor}'
        if odor_field in session_metadata:
            key = f'Odor {odor}'
            odorant_info[key] = session_metadata[odor_field]
    
    odorant_info_extension = None
    if odorant_info:
        odorant_info_extension = OdorantInfoExtension(
            name='odorant_info',
            **odorant_info
        )

    # Create experimental block extension
    block_info = {}
    for block in [1, 2, 3]:
        block_type_field = f'block{block}_type'
        if block_type_field in session_metadata:
            block_info[block_type_field] = ','.join(session_metadata[block_type_field])
    
    block_info_extension = None
    if block_info:
        block_info_extension = ExperimentalBlockExtension(
            name='block_info',
            **block_info
        )

    # Create preprocessed annotation extension
    preprocessed_annotations = {}
    
    # Pattern for matching SWR and control channel metadata
    annotation_patterns = [
        r'imec(\d+)_shank(\d+)_SWR_channel',
        r'imec(\d+)_best_SWR_channel',
        r'imec(\d+)_best_control_channel'
    ]
    
    # Find all keys in session_metadata that match our patterns
    for key in session_metadata:
        for pattern in annotation_patterns:
            if re.match(pattern, key):
                preprocessed_annotations[key] = session_metadata[key]
                break
    
    annotation_extension = None
    if preprocessed_annotations:
        annotation_extension = PreprocessedAnnotationExtension(
            name='preprocessed_annotations',
            **preprocessed_annotations
        )

    # Prepare the metadata dictionary for LabMetaDataExtension
    lab_metadata_dict = {
        'name': 'LabMetaData',
    }
    
    # Add probes if they exist
    if probe1_extension:
        lab_metadata_dict['probe1'] = probe1_extension
    
    if probe2_extension:
        lab_metadata_dict['probe2'] = probe2_extension
    
    # Add other extensions if they exist
    if odorant_info_extension:
        lab_metadata_dict['odorant_info'] = odorant_info_extension
    
    if block_info_extension:
        lab_metadata_dict['block_info'] = block_info_extension
    
    if annotation_extension:
        lab_metadata_dict['preprocessed_annotations'] = annotation_extension

    # Populate metadata extension 
    lab_metadata = LabMetaDataExtension(**lab_metadata_dict)

    # Add to file
    nwbfile.add_lab_meta_data(lab_metadata)


In [ ]:
from neuroconv.datainterfaces.ecephys.spikeglx.spikeglxdatainterface import SpikeGLXRecordingInterface
from neuroconv.tools.spikeinterface.spikeinterface import add_electrodes_info_to_nwbfile
from dateutil.tz import tzlocal

In [29]:
raw_session_dir = '/mnt/datasets/incoming/mvdm/OdorSequence/sourcedata/raw/M541/M541-2024-08-31_g0'
test_interface = SpikeGLXRecordingInterface(folder_path=Path(raw_session_dir), stream_id='imec0.ap')
raw_metadata = test_interface.get_metadata()

In [33]:
raw_metadata['NWBFile']['session_start_time'].astimezone()

datetime.datetime(2024, 8, 31, 18, 2, 46, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=72000), 'EDT'))

In [ ]:
# Create an NWB file
preprocessed_session_dir = '/mnt/datasets/incoming/mvdm/OdorSequence/sourcedata/preprocessed/M541/M541-2024-08-31'
preprocessed_metadata = mu.parse_expkeys(preprocessed_session_dir)
out_nwb = NWBFile(
    session_description="my first synthetic recording", #TODO: Can this be gleaned from exp keys?
    identifier='-'.join([preprocessed_metadata['subject'], session_metadata['date']]),
    # session_start_time=raw_metadata['NWBFile']['session_start_time'], # still needs timezone info, otherwise fails
    session_start_time=datetime.now(tzlocal()), #TODO: Get from .meta file
    experimenter=[
        session_metadata['experimenter']
    ],
    lab="vandermeerlab",
    institution="Dartmouth College",
    experiment_description="Head-fixed mouse presented with odor sequences",
    keywords=["ecephys", "exploration", "wanderlust"], #TODO: Needs editing
    )

In [40]:
isinstance(out_nwb, pynwb.NWBFile)

True

In [41]:
add_electrodes_info_to_nwbfile(recording=test_interface.recording_extractor, nwbfile=out_nwb, metadata=raw_metadata)

In [42]:
out_nwb

,location,group,group_name,channel_name,rel_x,contact_shapes,shank_ids,inter_sample_shift,contact_ids,rel_y
id,,,,,,,,,,
0,unknown,"NeuropixelsImec0Shank2 pynwb.ecephys.ElectrodeGroup at 0x139807459877520\nFields:\n description: A group representing probe/shank 'NeuropixelsImec0Shank2'.\n device: NeuropixelsImec0 pynwb.device.Device at 0x139807459880592\nFields:\n description: {""probe_type"": ""2013"", ""probe_type_description"": ""Unknown SpikeGLX probe type."", ""flex_part_number"": ""NPM_FLEX_01"", ""connected_base_station_part_number"": ""NP2_QBSC_00""}\n manufacturer: Imec\n\n location: unknown\n",NeuropixelsImec0Shank2,AP0,500.0,square,2,0.000000,s2e96,720.0
1,unknown,"NeuropixelsImec0Shank2 pynwb.ecephys.ElectrodeGroup at 0x139807459877520\nFields:\n description: A group representing probe/shank 'NeuropixelsImec0Shank2'.\n device: NeuropixelsImec0 pynwb.device.Device at 0x139807459880592\nFields:\n description: {""probe_type"": ""2013"", ""probe_type_description"": ""Unknown SpikeGLX probe type."", ""flex_part_number"": ""NPM_FLEX_01"", ""connected_base_station_part_number"": ""NP2_QBSC_00""}\n manufacturer: Imec\n\n location: unknown\n",NeuropixelsImec0Shank2,AP1,532.0,square,2,0.000000,s2e97,720.0
2,unknown,"NeuropixelsImec0Shank2 pynwb.ecephys.ElectrodeGroup at 0x139807459877520\nFields:\n description: A group representing probe/shank 'NeuropixelsImec0Shank2'.\n device: NeuropixelsImec0 pynwb.device.Device at 0x139807459880592\nFields:\n description: {""probe_type"": ""2013"", ""probe_type_description"": ""Unknown SpikeGLX probe type."", ""flex_part_number"": ""NPM_FLEX_01"", ""connected_base_station_part_number"": ""NP2_QBSC_00""}\n manufacturer: Imec\n\n location: unknown\n",NeuropixelsImec0Shank2,AP2,500.0,square,2,0.076923,s2e98,735.0
3,unknown,"NeuropixelsImec0Shank2 pynwb.ecephys.ElectrodeGroup at 0x139807459877520\nFields:\n description: A group representing probe/shank 'NeuropixelsImec0Shank2'.\n device: NeuropixelsImec0 pynwb.device.Device at 0x139807459880592\nFields:\n description: {""probe_type"": ""2013"", ""probe_type_description"": ""Unknown SpikeGLX probe type."", ""flex_part_number"": ""NPM_FLEX_01"", ""connected_base_station_part_number"": ""NP2_QBSC_00""}\n manufacturer: Imec\n\n location: unknown\n",NeuropixelsImec0Shank2,AP3,532.0,square,2,0.076923,s2e99,735.0


In [ ]:
# Add LFP electrodes and devices
device_labels = []
if os.path.exists(session_dir + "//imec0_clean_lfp.mat"):
    device_labels.append("imec0")
if os.path.exists(session_dir + "//imec1_clean_lfp.mat"):
    device_labels.append("imec1")
# add LFP electrodes table to nwb file
mnc.add_lfp_electrodes_to_nwb(session_dir, out_nwb, session_metadata, device_labels)
# add LFP traces to nwb file
mnc.add_lfp_data_to_nwb(session_dir, out_nwb, session_metadata, device_labels)

In [ ]:
# Add sorting electrodes and devices
device_labels = []
if os.path.exists(session_dir + "//clean_units_imec0.mat"):
    device_labels.append("imec0")
if os.path.exists(session_dir + "//clean_units_imec1.mat"):
    device_labels.append("imec1")
mnc.add_sorting_electrodes_to_nwb(session_dir, out_nwb, session_metadata, device_labels)
# add spike times, waveforms, and other information to nwb file
mnc.add_sorting_data_to_nwb(session_dir, out_nwb, session_metadata, device_labels)

In [ ]:
# Add behavioral epoohs
mnc.add_intervals_to_nwb(out_nwb, session_metadata)

In [ ]:
# Add lab metadata using the new extension
add_lab_metadata_to_nwb(session_dir, out_nwb, session_metadata)

### Write the actual NWB File

In [ ]:
io = NWBHDF5IO("E:\\odor-pixels\\fromHector\\NoReward\\M541\\M541-2024-08-31\\test_metadata.nwb", mode='w')
io.write(out_nwb)
io.close() # This is crtitcal and nwbinspector won't work without it

In [ ]:
out_nwb

In [ ]:
io = NWBHDF5IO("E:\\odor-pixels\\fromHector\\NoReward\\M541\\M541-2024-08-31\\test_2024-08-31.nwb", mode='r')

In [ ]:
out_nwb2 = io.read()
out_nwb2